In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient

In [4]:
AZURE_FORM_RECOGNIZER_ENDPOINT = "https://form-service-rtw.cognitiveservices.azure.com/"
AZURE_FORM_RECOGNIZER_KEY = "6752434521604f128f1c6f62236050df"

endpoint = AZURE_FORM_RECOGNIZER_ENDPOINT
key = AZURE_FORM_RECOGNIZER_KEY

form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [13]:
with open("../material_preparation_step/digital_id_template/ca-dl-james-webb.png", "rb") as f:
    poller = form_recognizer_client.begin_recognize_content(form=f)
form_pages = poller.result()

In [14]:
form_pages

[FormPage(page_number=1, text_angle=0.0, width=507.0, height=310.0, unit=pixel, tables=[], lines=[FormLine(text=California, bounding_box=[Point(x=27.0, y=7.0), Point(x=195.0, y=7.0), Point(x=195.0, y=39.0), Point(x=27.0, y=39.0)], words=[FormWord(text=California, bounding_box=[Point(x=28.0, y=8.0), Point(x=192.0, y=8.0), Point(x=191.0, y=40.0), Point(x=28.0, y=40.0)], confidence=0.992, page_number=1, kind=word)], page_number=1, kind=line, appearance=TextAppearance(style_name=other, style_confidence=0.878)), FormLine(text=DRIVER LICENSE, bounding_box=[Point(x=240.0, y=10.0), Point(x=382.0, y=10.0), Point(x=382.0, y=27.0), Point(x=240.0, y=27.0)], words=[FormWord(text=DRIVER, bounding_box=[Point(x=240.0, y=11.0), Point(x=299.0, y=11.0), Point(x=299.0, y=28.0), Point(x=240.0, y=28.0)], confidence=0.991, page_number=1, kind=word), FormWord(text=LICENSE, bounding_box=[Point(x=305.0, y=11.0), Point(x=378.0, y=11.0), Point(x=377.0, y=27.0), Point(x=305.0, y=28.0)], confidence=0.996, page_numb

In [15]:
def get_id_card_details(identity_card):
    first_name = identity_card.fields.get("FirstName")
    if first_name:
        print("First Name: {} has confidence: {}".format(first_name.value, first_name.confidence))
    last_name = identity_card.fields.get("LastName")
    if last_name:
        print("Last Name: {} has confidence: {}".format(last_name.value, last_name.confidence))

In [16]:
get_id_card_details(form_pages[0])

AttributeError: 'FormPage' object has no attribute 'fields'

In [19]:
form_pages[0]

FormPage(page_number=1, text_angle=0.0, width=507.0, height=310.0, unit=pixel, tables=[], lines=[FormLine(text=California, bounding_box=[Point(x=27.0, y=7.0), Point(x=195.0, y=7.0), Point(x=195.0, y=39.0), Point(x=27.0, y=39.0)], words=[FormWord(text=California, bounding_box=[Point(x=28.0, y=8.0), Point(x=192.0, y=8.0), Point(x=191.0, y=40.0), Point(x=28.0, y=40.0)], confidence=0.992, page_number=1, kind=word)], page_number=1, kind=line, appearance=TextAppearance(style_name=other, style_confidence=0.878)), FormLine(text=DRIVER LICENSE, bounding_box=[Point(x=240.0, y=10.0), Point(x=382.0, y=10.0), Point(x=382.0, y=27.0), Point(x=240.0, y=27.0)], words=[FormWord(text=DRIVER, bounding_box=[Point(x=240.0, y=11.0), Point(x=299.0, y=11.0), Point(x=299.0, y=28.0), Point(x=240.0, y=28.0)], confidence=0.991, page_number=1, kind=word), FormWord(text=LICENSE, bounding_box=[Point(x=305.0, y=11.0), Point(x=378.0, y=11.0), Point(x=377.0, y=27.0), Point(x=305.0, y=28.0)], confidence=0.996, page_numbe

In [24]:
for word in form_pages[0].lines:
    print(word.text)

California
DRIVER LICENSE
DL D4357789
CLASS C
EXP
01/10/2026
END
LN
Webb
FN
James
DOB 12/15/1970
RSTR
DONOR
SEX M
HAIR BRN
EYES BLK
HGT 5' 11"
WGT 192 1b
ISS
D
08/08/2020


In [27]:
form_pages[0].lines[7]

FormLine(text=LN, bounding_box=[Point(x=177.0, y=116.0), Point(x=197.0, y=114.0), Point(x=197.0, y=127.0), Point(x=177.0, y=128.0)], words=[FormWord(text=LN, bounding_box=[Point(x=177.0, y=115.0), Point(x=191.0, y=114.0), Point(x=192.0, y=127.0), Point(x=177.0, y=128.0)], confidence=0.986, page_number=1, kind=word)], page_number=1, kind=line, appearance=TextAppearance(style_name=other, style_confidence=0.878))

In [34]:
content_url = "https://raw.githubusercontent.com/udacity/cd0461-building-computer-vision-solutions-with-azure-exercises/main/resources/ca-dl-james-webb.png.png"

In [35]:
id_content_from_url = form_recognizer_client.begin_recognize_identity_documents_from_url(content_url)

HttpResponseError: (InvalidImage) The input data is not a valid image or password protected.
Code: InvalidImage
Message: The input data is not a valid image or password protected.

In [ ]:
collected_id_cards = id_content_from_url.result()

In [31]:
collected_id_cards

[RecognizedForm(form_type=prebuilt:idDocument:driverLicense, fields={'Address': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=1234 Circle Ave, Apt 123 San Mateo, CA, 94401, bounding_box=[Point(x=180.0, y=149.0), Point(x=329.0, y=149.0), Point(x=329.0, y=182.0), Point(x=180.0, y=182.0)], field_elements=None), name=Address, value='1234 Circle Ave, Apt 123 San Mateo, CA, 94401', confidence=0.657), 'CountryRegion': FormField(value_type=countryRegion, label_data=None, value_data=None, name=CountryRegion, value='USA', confidence=0.99), 'DateOfBirth': FormField(value_type=date, label_data=None, value_data=FieldData(page_number=1, text=01/01/1990, bounding_box=[Point(x=208.0, y=182.0), Point(x=285.0, y=181.0), Point(x=286.0, y=196.0), Point(x=208.0, y=196.0)], field_elements=None), name=DateOfBirth, value=datetime.date(1990, 1, 1), confidence=0.995), 'DateOfExpiration': FormField(value_type=date, label_data=None, value_data=FieldData(page_number=1, text

In [32]:
def get_id_card_details(identity_card):
    first_name = identity_card.fields.get("FirstName")
    if first_name:
        print("First Name: {} has confidence: {}".format(first_name.value, first_name.confidence))
    last_name = identity_card.fields.get("LastName")
    if last_name:
        print("Last Name: {} has confidence: {}".format(last_name.value, last_name.confidence))
    document_number = identity_card.fields.get("DocumentNumber")
    if document_number:
        print("Document Number: {} has confidence: {}".format(document_number.value, document_number.confidence))
    dob = identity_card.fields.get("DateOfBirth")
    if dob:
        print("Date of Birth: {} has confidence: {}".format(dob.value, dob.confidence))
    doe = identity_card.fields.get("DateOfExpiration")
    if doe:
        print("Date of Expiration: {} has confidence: {}".format(doe.value, doe.confidence))
    sex = identity_card.fields.get("Sex")
    if sex:
        print("Sex: {} has confidence: {}".format(sex.value, sex.confidence))
    address = identity_card.fields.get("Address")
    if address:
        print("Address: {} has confidence: {}".format(address.value, address.confidence))
    country_region = identity_card.fields.get("CountryRegion")
    if country_region:
        print("Country/Region: {} has confidence: {}".format(country_region.value, country_region.confidence))
    region = identity_card.fields.get("Region")
    if region:
        print("Region: {} has confidence: {}".format(region.value, region.confidence))

In [33]:
get_id_card_details(collected_id_cards[0])

First Name: AVKASH CHAUHAN has confidence: 0.781
Last Name: CHAUHAN has confidence: 0.895
Document Number: D1234578 has confidence: 0.995
Date of Birth: 1990-01-01 has confidence: 0.995
Date of Expiration: 2025-01-01 has confidence: 0.99
Sex: M has confidence: 0.985
Address: 1234 Circle Ave, Apt 123 San Mateo, CA, 94401 has confidence: 0.657
Country/Region: USA has confidence: 0.99
Region: California has confidence: 0.983
